# Grand Question 1:
## 1. Which airport has the worst delays? How did you choose to define “worst”? As part of your answer include a table that lists the total number of flights, total number of delayed flights, proportion of delayed flights, and average delay time in hours, for each airport.

In [2]:
# Imports
import pandas as pd
import altair as alt
import numpy as np
import urllib3
import json


# Loading data
url = 'https://github.com/byuidatascience/data4missing/raw/master/data-raw/flights_missing/flights_missing.json'
df = pd.read_json(url)

http = urllib3.PoolManager()
response = http.request('GET', url)
flights_json = json.loads(response.data.decode('utf-8'))
flights = pd.json_normalize(flights_json)

# df

# Check for nulls with `df.isna().sum()`
# From df.isna().sum(), I see there are missing values, so I will instead of 
# deleting them, fill them with the mean of the column.
df = df.fillna(df.mean())
df_q1 = df.copy()


/var/folders/0q/79csbplj4f75d38xhbr9wfp80000gn/T/ipykernel_20118/1755514984.py:23: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean())


In [3]:
# Total flights per airport
total_flights_per_airport = df_q1.groupby('airport_code').num_of_flights_total.sum().sort_values(ascending=False)
print('Total flights per airport\n',total_flights_per_airport, '\n')

# Total delays per airport
total_delays_per_airport = df_q1.groupby('airport_code').num_of_delays_total.sum().sort_values(ascending=False)
print('Total delays per airport\n', total_delays_per_airport, '\n')

# Add proportion of delayed flights column
df_q1['proportion_of_delayed_flights'] = df_q1.apply(lambda x: x['num_of_flights_total']/x['num_of_delays_total'], axis=1)

# Average delay length
df_q1['delay_length_average'] = df_q1.apply(lambda x: x['minutes_delayed_total']/x['num_of_delays_total'], axis=1)

# Subset
df_q1 = df_q1[['airport_code','num_of_flights_total',
         'num_of_delays_total',
         'proportion_of_delayed_flights',
         'minutes_delayed_total',
         'delay_length_average']]

# Individualize the airports
df_q1_ORD = df_q1.query('airport_code == "ORD"')
df_q1_SFO = df_q1.query('airport_code == "SFO"')
df_q1_IAD = df_q1.query('airport_code == "IAD"')
df_q1_ATL = df_q1.query('airport_code == "ATL"')
df_q1_DEN = df_q1.query('airport_code == "DEN"')
df_q1_SLC = df_q1.query('airport_code == "SLC"')
df_q1_SAN = df_q1.query('airport_code == "SAN"')

# Get delay average in minutes
df_q1_ORD_delays = df_q1_ORD['delay_length_average'].sum()/len(df_q1_ORD)
df_q1_SFO_delays = df_q1_SFO['delay_length_average'].sum()/len(df_q1_SFO)
df_q1_IAD_delays = df_q1_IAD['delay_length_average'].sum()/len(df_q1_IAD)
df_q1_ATL_delays = df_q1_ATL['delay_length_average'].sum()/len(df_q1_ATL)
df_q1_DEN_delays = df_q1_DEN['delay_length_average'].sum()/len(df_q1_DEN)
df_q1_SLC_delays = df_q1_SLC['delay_length_average'].sum()/len(df_q1_SLC)
df_q1_SAN_delays = df_q1_SAN['delay_length_average'].sum()/len(df_q1_SAN)

# Delay Percentage/Proportion
delay_percentage = (total_delays_per_airport/total_flights_per_airport).sort_values(ascending=False)
print('Delay Percentage\n',  delay_percentage)
print()

# Print delay averages
print(f'ORD average delay is {df_q1_ORD_delays:.0f} minutes')
print(f'SFO average delay is {df_q1_SFO_delays:.0f} minutes')
print(f'IAD average delay is {df_q1_IAD_delays:.0f} minutes')
print(f'ATL average delay is {df_q1_ATL_delays:.0f} minutes')
print(f'DEN average delay is {df_q1_DEN_delays:.0f} minutes')
print(f'SLC average delay is {df_q1_SLC_delays:.0f} minutes')
print(f'SAN average delay is {df_q1_SAN_delays:.0f} minutes')
print()

#Turn delay_percentage to dict
delay_pct_dict = delay_percentage.to_dict()

# Worst delay airport
most_by = 4430047-3597588
atl_pct = 902443/4430047


Total flights per airport
 airport_code
ATL    4430047
ORD    3597588
DEN    2513974
SFO    1630945
SLC    1403384
SAN     917862
IAD     851571
Name: num_of_flights_total, dtype: int64 

Total delays per airport
 airport_code
ATL    902443
ORD    830825
DEN    468519
SFO    425604
SLC    205160
SAN    175132
IAD    168467
Name: num_of_delays_total, dtype: int64 

Delay Percentage
 airport_code
SFO    0.260955
ORD    0.230939
ATL    0.203710
IAD    0.197831
SAN    0.190804
DEN    0.186366
SLC    0.146189
dtype: float64

ORD average delay is 66 minutes
SFO average delay is 61 minutes
IAD average delay is 60 minutes
ATL average delay is 59 minutes
DEN average delay is 53 minutes
SLC average delay is 49 minutes
SAN average delay is 47 minutes



# Grand Question 2:
### What is the worst month to fly if you want to avoid delays? Include one chart to help support your answer, with the x-axis ordered by month. You also need to explain and justify how you chose to handle the missing Month data.¶

In [4]:
df_q1.isna().sum()

airport_code                     0
num_of_flights_total             0
num_of_delays_total              0
proportion_of_delayed_flights    0
minutes_delayed_total            0
delay_length_average             0
dtype: int64

In [15]:
# Loading data
df_q2 = df.copy()

# Convert month column to numeric
## After examining the data, I should make the `n/a` data to match the above month
ls = []

for i in df_q2['month']:
    ls.append(i)

x = 0

for i in range(len(ls)):
   if ls[i] == 'n/a':
     ls[i] = ls[i-1]
     x+=1
print(x) #27
df_q2['month'] = ls

# Convert string month to numeric
def monthToNum(shortMonth):
    return {
        'January': 1,
        'Febuary': 2,
        'March': 3,
        'April': 4,
        'May': 5,
        'June': 6,
        'July': 7,
        'August': 8,
        'September': 9, 
        'October': 10,
        'November': 11,
        'December': 12
    }[shortMonth]

# Convert the data to numeric data
df_q2['month'] = df_q2['month'].apply(lambda x: monthToNum(x))

# df_q2['month']= df_q2['month'].sort_index(key=lambda x: pd.to_datetime(x, format="%B"))

# Organize with groupby month
total_delays_per_month = df_q2.groupby('month').num_of_delays_total.sum().sort_values(ascending=False)

# Convert to DF for charting
dfx = pd.DataFrame([total_delays_per_month]).T

# Fix indexing issue
dfx['month'] = dfx.index
dfx.reset_index(drop=True, inplace=True)

# Format the way I want
dfx = dfx[['month', 'num_of_delays_total']]

# Chart
total_delays_per_month_chart = (
    alt.Chart(dfx)
    .encode(
        x='month',
        y='num_of_delays_total')
    .mark_bar()
    .properties(width=500, title="June is the worst month for fly "))

# Results
print("As we can see in the chart below, June is the worst month for delays at the airports observed.\n")

# SAVE CHART
# total_delays_per_month_chart.save('total_delays_per_month_chart.png')
total_delays_per_month_chart

27
As we can see in the chart below, June is the worst month for delays at the airports observed.



alt.Chart(...)

In [14]:
# Loading data
df_q2 = df.copy()

# Convert month column to numeric
## After examining the data, I should make the `n/a` data to match the above month
ls = []

for i in df_q2['month']:
    ls.append(i)

x = 0

for i in range(len(ls)):
   if ls[i] == 'n/a':
     ls[i] = ls[i-1]
     x+=1
print(x) #27
df_q2['month'] = ls

# Convert string month to numeric
def monthToNum(shortMonth):
    return {
        'January': 1,
        'Febuary': 2,
        'March': 3,
        'April': 4,
        'May': 5,
        'June': 6,
        'July': 7,
        'August': 8,
        'September': 9, 
        'October': 10,
        'November': 11,
        'December': 12
    }[shortMonth]

# Convert the data to numeric data
# df_q2['month'] = df_q2['month'].apply(lambda x: monthToNum(x))

df_q2['month']= df_q2['month'].sort_index(key=lambda x: pd.to_datetime(x, format="%B"))

# Organize with groupby month
total_delays_per_month = df_q2.groupby('month').num_of_delays_total.sum().sort_values(ascending=False)

# Convert to DF for charting
dfx = pd.DataFrame([total_delays_per_month]).T

# Fix indexing issue
dfx['month'] = dfx.index
dfx.reset_index(drop=True, inplace=True)

# Format the way I want
dfx = dfx[['month', 'num_of_delays_total']]

# Chart
total_delays_per_month_chart = (
    alt.Chart(dfx)
    .encode(
        x='month',
        y='num_of_delays_total')
    .mark_bar()
    .properties(width=500, title="June is the worst month for fly "))

# Results
print("As we can see in the chart below, June is the worst month for delays at the airports observed.\n")

# SAVE CHART
# total_delays_per_month_chart.save('total_delays_per_month_chart.png')
total_delays_per_month_chart

27
As we can see in the chart below, June is the worst month for delays at the airports observed.



alt.Chart(...)

### 3. According to the BTS website the Weather category only accounts for severe weather delays. Other “mild” weather delays are included as part of the NAS category and the Late-Arriving Aircraft category. Calculate the total number of flights delayed by weather (either severe or mild) using these two rules:


In [4]:
def missing_checks(df, column ):
    out1 = df[column].isnull().sum(axis = 0)
    out2 = df[column].describe()
    out3 = df[column].describe(exclude=np.number)
    print('\n\n\n')
    print('Checking column' + column)
    print('\n')
    print('Missing summary')
    print(out1)
    print('\n')
    print("Numeric summaries")
    print(out2)
    print('\n')
    print('Non Numeric summaries')
    print(out3)

missing_checks(flights, 'num_of_delays_nas')
missing_checks(flights, 'num_of_delays_late_aircraft')
missing_checks(flights, 'num_of_delays_weather')    





Checking columnnum_of_delays_nas


Missing summary
0


Numeric summaries
count     924.000000
mean     1376.467532
std      1348.719957
min        61.000000
25%       357.750000
50%       960.000000
75%      1869.250000
max      8704.000000
Name: num_of_delays_nas, dtype: float64


Non Numeric summaries
count     924.000000
mean     1376.467532
std      1348.719957
min        61.000000
25%       357.750000
50%       960.000000
75%      1869.250000
max      8704.000000
Name: num_of_delays_nas, dtype: float64




Checking columnnum_of_delays_late_aircraft


Missing summary
0


Numeric summaries
count     924.000000
mean     1017.844156
std       853.942405
min      -999.000000
25%       488.750000
50%       804.000000
75%      1473.750000
max      3969.000000
Name: num_of_delays_late_aircraft, dtype: float64


Non Numeric summaries
count     924.000000
mean     1017.844156
std       853.942405
min      -999.000000
25%       488.750000
50%       804.000000
75%      1473.750000
max    

In [6]:
flights.num_of_delays_weather.describe()

count    924.000000
mean     100.971861
std      103.584998
min        3.000000
25%       34.750000
50%       66.000000
75%      129.000000
max      812.000000
Name: num_of_delays_weather, dtype: float64

In [11]:
weather = (flights.assign(
    severe = flights.num_of_delays_weather, # no missing
    nodla_nona = lambda x: (x.num_of_delays_late_aircraft
        .replace(-999, np.nan)), #missing is -999
    mild_late = lambda x: x.nodla_nona.fillna(x.nodla_nona.mean())*0.3,
    mild = np.where(
        flights.month.isin(['April', 'May', 'June', 'July', 'August']), 
            flights.num_of_delays_nas*0.4,
            flights.num_of_delays_nas*0.65),
    weather = lambda x: x.severe + x.mild_late + x.mild,
    proportion_weather_delay = lambda x: x.weather / x.num_of_delays_total,
    proportion_weather_total = lambda x:  x.weather / x.num_of_flights_total)
    .filter(['airport_code','month','year', 'severe','mild', 'mild_late',
    'weather', 'proportion_weather_total', 
    'proportion_weather_delay', 'num_of_flights_total', 'num_of_delays_total']))

In [12]:
weather

,airport_code,month,year,severe,mild,mild_late,weather,proportion_weather_total,proportion_weather_delay,num_of_flights_total,num_of_delays_total
0,ATL,January,2005.0,448,2988.70,332.731222,3769.431222,0.107551,0.451159,35048,8355
1,DEN,January,2005.0,233,607.75,278.400000,1119.150000,0.088212,0.354948,12687,3153
2,IAD,January,2005.0,61,581.75,317.400000,960.150000,0.077550,0.395123,12381,2430
3,ORD,January,2005.0,306,3519.75,676.500000,4502.250000,0.159688,0.490548,28194,9178
4,SAN,January,2005.0,56,414.70,204.000000,674.700000,0.092640,0.345645,7283,1952
...,...,...,...,...,...,...,...,...,...,...,...
919,IAD,December,2015.0,17,39.65,54.900000,111.550000,0.039854,0.251806,2799,443
920,ORD,December,2015.0,180,886.60,526.500000,1593.100000,0.062308,0.376352,25568,4233
921,SAN,n/a,2015.0,37,166.40,181.800000,385.200000,0.061820,0.278525,6231,1383
922,SFO,December,2015.0,147,1541.80,354.000000,2042.800000,0.147676,0.457514,13833,4465
